# Polyiamonds

Working ... https://puzzler.sourceforge.net/docs/FAQ.html#

A Puzzle grid selector tool is also provided [here](https://smilingwayne.github.io/PuzzleTools/)!



## Import dependencies

In [9]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon
from collections import defaultdict

## Coordinates

In [ ]:
def get_rectified_coord(x, y, z, theta):
    """Get Rectified Coordinates via (x, y, z) and theta (in degree form).
    """
    radian = np.deg2rad(theta)
    if z == 0:
        return [(x + y * np.cos(radian), y * np.sin(radian)), 
                (x + y * np.cos(radian) + 1, y * np.sin(radian)),
                (x + y * np.cos(radian) + np.cos(radian), (y + 1) * np.sin(radian))
               ]
    elif z == 1:
        return [
            (x + y * np.cos(radian) + np.cos(radian), (y + 1) * np.sin(radian)),
            (x + y * np.cos(radian) + np.cos(radian) + 1, (y + 1) * np.sin(radian)),
            (x + y * np.cos(radian) + 1, y * np.sin(radian))
        ]
    else: 
        print(f"Invalid z coord (only 1 or 0)! Found {z}!")
        return []

# The neighbors of the triangle at coordinates (x, y, 0) are:
# {(x, y, 1), (x-1, y, 1), (x, y-1, 1)}
# The neighbors of the triangle at coordinates (x, y, 1) are:
# {(x, y, 0), (x+1, y, 0), (x, y+1, 0)}

def round_point(point):
    return tuple(map(lambda x: round(x, 5), point))

def round_coords(segments):
    return list(map(lambda line: (round_point(line[0]), round_point(line[1])), segments))

def get_border(coords, theta): 
    candidates = set()
    removed_edge = set()
    final_edges = list()
    for (x, y, z) in coords: 
        candidates.add(f"{x}_{y}_{z}")
        if z == 0: 
            if f"{x}_{y}_{1}" in candidates: 
                removed_edge.add(f"{x}_{y}_{0}_{1}") 
                removed_edge.add(f"{x}_{y}_{1}_{2}") 

            if f"{x - 1}_{y}_{1}" in candidates:  
                removed_edge.add(f"{x}_{y}_{0}_{2}")
                removed_edge.add(f"{x - 1}_{y}_{1}_{1}")

            if f"{x}_{y - 1}_{1}" in candidates: 
                removed_edge.add(f"{x}_{y}_{0}_{0}")
                removed_edge.add(f"{x}_{y - 1}_{1}_{0}")

        elif z == 1: 
            if f"{x}_{y}_{0}" in candidates: 
                removed_edge.add(f"{x}_{y}_{1}_{2}")
                removed_edge.add(f"{x}_{y}_{0}_{1}")
                
            if f"{x + 1}_{y}_{0}" in candidates: 
                removed_edge.add(f"{x}_{y}_{1}_{1}")
                removed_edge.add(f"{x + 1}_{y}_{0}_{2}")

            if f"{x}_{y + 1}_{0}" in candidates: 
                removed_edge.add(f"{x}_{y}_{1}_{0}")
                removed_edge.add(f"{x}_{y + 1}_{0}_{0}")
    for (x, y, z) in coords: 
        pos = get_rectified_coord(x, y, z, theta)
        for i in range(3): 
            if f"{x}_{y}_{z}_{i}" in removed_edge: 
                continue 
            else:
                final_edges.append([pos[i % 3], pos[(i + 1) % 3]])
    return final_edges 
            

def organize_segments(segments):
    # 将每个线段转化为点对字典
    point_dict = {}
    for (x1, y1), (x2, y2) in segments:
        point_dict.setdefault((x1, y1), []).append((x2, y2))
        point_dict.setdefault((x2, y2), []).append((x1, y1))

    # 从其中一个点开始重新构建闭合路径
    start_point = segments[0][0]
    current_point = start_point
    organized_path = [start_point]

    while len(organized_path) != len(segments) + 1:
        # 从当前点选择下一个点
        next_point = point_dict[current_point][0]  # 选择尚未访问的下一个点
        # 移动到下一个点
        point_dict[current_point].remove(next_point)
        point_dict[next_point].remove(current_point)
        organized_path.append(next_point)
        current_point = next_point

    return organized_path

def clear_padding(coords):
    """Adjust the shape to border.

    Args:
        coords (_type_): _description_

    Returns:
        _type_: _description_
    """
    min_x = min(x for x, _, _ in coords) 
    min_y = min(y for _, y, _ in coords) 
    offset_x = min_x - 0
    offset_y = min_y - 0 

    if offset_x > 0 or offset_y > 0:
        new_coords = []
        for (x, y, z) in coords: 
            new_coords.append((x - offset_x, y - offset_y, z))
        return new_coords 
    else:
        return coords

def flap_120(coords):
    max_x = max(x for x, _, _ in coords)
    max_y = max(y for _, y, _ in coords)
    axis_ = max(max_x, max_y)
    flap_coords = []
    for (x, y, z) in coords: 
        if z == 1: 
            flap_coords.append((axis_ - y, axis_ - x, 0))
        elif z == 0:
            flap_coords.append((axis_ - y, axis_ - x, 1))
        else:
            print(f"Invalid coords: {z}, should be 0 or 1.")
    return clear_padding(flap_coords) 

def flap_90(coords):
    max_xy = max([ x + y if z == 0 else x + y + 1 for (x, y, z) in coords])
    new_coords = []
    for (x, y, z) in coords: 
        if z == 0: 
            new_coords.append((max_xy - x - y, y, z))
        else:
            new_coords.append((max_xy - x - y - 1, y, z))
    return clear_padding(new_coords)

def flap_30(coords):
    flap_coords = [] 
    for (x, y, z) in coords: 
        flap_coords.append((y, x, z)) 
    return clear_padding(flap_coords)

def all_possible_rotations(coords, opts = ["flap_90", "flap_120", "flap_30"]): 
    final_coords = [coords]
    for idx, opt in enumerate(opts): 
        if opt == "flap_90":
            next_coords = flap_90(coords)
        elif opt == "flap_120": 
            next_coords = flap_120(coords)
        elif opt == "flap_30":
            next_coords = flap_30(coords)
        new_coords = all_possible_rotations(next_coords, opts[: idx] + opts[idx + 1:])
        final_coords = final_coords +  new_coords
    return final_coords

def remove_dup(coords):
    seen = set()
    result = []
    for sublist in coords:
        # 将子列表中的元组排序并转换为元组作为唯一标识
        sorted_sublist = sorted(sublist)
        key = tuple(sorted_sublist)
        if key not in seen:
            seen.add(key)
            result.append(sublist)
    return result

def plot_polyiamond(coords_list):
    
    theta = 60
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    
    for idx, coord in enumerate(coords_list):
        temp_coords = get_border(coord, theta)
        temp_coords = round_coords(temp_coords)
        temp_coords = organize_segments(temp_coords)

        polygon = Polygon(temp_coords, closed=True, fill=True, edgecolor='black', facecolor='skyblue', alpha=0.5)
        ax.add_patch(polygon)
        
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_xlim(-1, 15)
    ax.set_ylim(-1, 15)
    ax.grid()
    ax.set_title('Filled Polygon from Line Segments')
    plt.show()

if __name__ == "__main__": 
    triangles = [
        # (0, 0, 0), (0, 0, 1), (1, 0, 0), (1, 0, 1), (2, 0, 0), (2, 0, 1),(0, 1, 0), (0, 1, 1),(0, 2, 0), (0, 2, 1),(1, 2, 0), (1, 2, 1)
        # rhomboid
        # (0, 0, 0), (0, 0, 1), 
        # (1, 0, 0), (1, 0, 1),
        # (2, 0, 0), (0, 1, 0)
        # club: 
        # (0, 0, 1), 
        # (1, 0, 0), (1, 0, 1), 
        # (2, 0, 0), (2, 0, 1), 
        # (0, 1, 0)
        # Crown:
        # (0, 0, 1), 
        # (1, 0, 0), (1, 0, 1), 
        # (2, 0, 0), (2, 0, 1), 
        # (1, 1, 0)
        # Sphinx: 
        # (0, 0, 0), (0, 0, 1), 
        # (1, 0, 0), (1, 0, 1),
        # (2, 0, 0), 
        # (0, 1, 0)
        # Snake: 
        # (0, 0, 1), 
        # (0, 1, 0), (0, 1, 1), 
        # (1, 1, 0), (1, 1, 1),
        # (1, 2, 0)
        # Yacht: 
        # (0, 0, 0), (0, 0, 1), 
        # (1, 0, 0), (1, 0, 1), 
        # (0, 1, 0), (1, 1, 0)
        # Bat:
        # (0, 0, 0), (0, 0, 1), 
        # (1, 0, 0), (1, 0, 1), 
        # (1, 1, 0), (1, 1, 1)
        # Pistol:
        # (0, 0, 1), 
        # (0, 1, 0), (0, 1, 1), 
        # (1, 1, 0), (1, 1, 1),
        # (0, 2, 0)
        # Lobster:
        # (0, 0, 1), 
        # (0, 1, 0), (0, 1, 1),
        # (0, 2, 0),
        # (1, 0, 1), 
        # (1, 1, 0)
        # Hook:
        # (0, 0, 0), (0, 0, 1),
        # (0, 1, 0), (0, 1, 1),
        # (1, 0, 1), (1, 1, 0)
        # Hexagon:
        # (0, 0, 1), 
        # (0, 1, 0), (0, 1, 1),
        # (1, 0, 0), (1, 0, 1), 
        # (1, 1, 0)
        # Butterfly:
        (0, 1, 1), 
        (1, 0, 1), (1, 0, 0), 
        (2, 0, 0), 
        (1, 1, 0), (1, 1, 1)
        
        # For Test:
        # (0, 0, 1),
        # (1, 0, 0), (1, 0, 1), 
        # (0, 1, 0), (0, 1, 1),
        # (1, 1, 0), (2, 0, 0)
    ]
    all_coords = all_possible_rotations(triangles)

    all_coords = remove_dup(all_coords)

    # for x in all_coords: 
    #     print(x)
    #     plot_polyiamond([x])
    # plot_polyiamond(all_coords)
    

In [13]:
from collections import defaultdict 
from ortools.sat.python import cp_model as cp

In [14]:
def generate_grid(name):
    if name == "4x9":
        return {
            "x": 9, 
            "y": 4,
            "grid": ",".join([f"{a}_{b}_{c}" for a in range(9) for b in range(4) for c in range(2)])
        }
    if name == "6x6":
        return {
            "x": 6,
            "y": 6,
            "grid": ",".join([f"{a}_{b}_{c}" for a in range(6) for b in range(6) for c in range(2)])
        }
    

class Grid:
    def __init__(self, grid_dict):
        self.width = grid_dict['x']
        self.height = grid_dict['y']
        self.grid_str = grid_dict['grid'] 
        self.positions = set(tuple(int(part) for part in coord.split('_')) for coord in self.grid_str.split(','))
        
    def get_all_feasible_pos(self, shape): 
        candidates = defaultdict(list)
        for x_1 in range(self.width):
            for y_1 in range(self.height):
                if self.check_fit(shape, x_1, y_1):
                    candidates[x_1, y_1].append(shape)
        return candidates

    def check_fit(self, shape, x_, y_):
        """Check if "shape" can be fit into position (x_, y_).

        Args:
            shape (_type_): _description_
            x_ (_type_): _description_
            y_ (_type_): _description_

        Returns:
            _type_: _description_
        """
        for (x, y, z) in shape: 
            if (x + x_, y + y_, z) not in self.positions:
                return False 
        return True
if __name__ == "__main__":
    test_grid = generate_grid("4x9")
    dummy_grid = Grid(test_grid)
    # print(dummy_grid.check_fit(triangles, 0, 0))
    test_candidates = dummy_grid.get_all_feasible_pos(triangles)
    print(test_candidates)


defaultdict(<class 'list'>, {(0, 0): [[(0, 1, 1), (1, 0, 1), (1, 0, 0), (2, 0, 0), (1, 1, 0), (1, 1, 1)]], (0, 1): [[(0, 1, 1), (1, 0, 1), (1, 0, 0), (2, 0, 0), (1, 1, 0), (1, 1, 1)]], (0, 2): [[(0, 1, 1), (1, 0, 1), (1, 0, 0), (2, 0, 0), (1, 1, 0), (1, 1, 1)]], (1, 0): [[(0, 1, 1), (1, 0, 1), (1, 0, 0), (2, 0, 0), (1, 1, 0), (1, 1, 1)]], (1, 1): [[(0, 1, 1), (1, 0, 1), (1, 0, 0), (2, 0, 0), (1, 1, 0), (1, 1, 1)]], (1, 2): [[(0, 1, 1), (1, 0, 1), (1, 0, 0), (2, 0, 0), (1, 1, 0), (1, 1, 1)]], (2, 0): [[(0, 1, 1), (1, 0, 1), (1, 0, 0), (2, 0, 0), (1, 1, 0), (1, 1, 1)]], (2, 1): [[(0, 1, 1), (1, 0, 1), (1, 0, 0), (2, 0, 0), (1, 1, 0), (1, 1, 1)]], (2, 2): [[(0, 1, 1), (1, 0, 1), (1, 0, 0), (2, 0, 0), (1, 1, 0), (1, 1, 1)]], (3, 0): [[(0, 1, 1), (1, 0, 1), (1, 0, 0), (2, 0, 0), (1, 1, 0), (1, 1, 1)]], (3, 1): [[(0, 1, 1), (1, 0, 1), (1, 0, 0), (2, 0, 0), (1, 1, 0), (1, 1, 1)]], (3, 2): [[(0, 1, 1), (1, 0, 1), (1, 0, 0), (2, 0, 0), (1, 1, 0), (1, 1, 1)]], (4, 0): [[(0, 1, 1), (1, 0, 1), (1,

In [ ]:
def get_polyiamonds_by_size(sizes):
    sizes = set(sizes)
    shapes = []
    shapes_name = []
    for size in sizes:
        if size == 7: 
            shapes += [
                # A7
                [(0,0,1), (0,1,1), (0,2,0), (1,0,0), (1,0,1), (1,1,0), (1,1,1)], 
                # B7 
                [(0,0,1), (0,1,1), (1,0,0), (1,0,1), (1,1,0), (1,1,1), (2,0,0)],
                # C7
                [(0,0,1), (0,1,0), (0,1,1), (1,1,0), (1,1,1), (2,0,1), (2,1,0)],
                # D7 
                [(0,0,0), (0,0,1), (0,1,0), (0,1,1), (1,0,0), (1,0,1), (1,1,0)], 
                # E7
                [(0,0,0), (0,0,1), (0,1,0), (0,1,1), (0,2,0), (0,2,1), (1,1,0)],
                # F7
                [(0,0,1), (0,1,0), (0,1,1), (0,2,0), (0,2,1), (1,0,0), (1,1,0)],
                # G7
                [(0,0,0), (0,0,1), (0,1,0), (0,1,1), (1,0,0), (1,0,1), (2,0,0)],
                # H7
                [(0,0,0), (0,0,1), (1,0,0), (1,0,1), (1,1,0), (1,1,1), (2,0,0)],
                # I7
                [(0,0,0), (0,0,1), (1,0,0), (1,0,1), (2,0,0), (2,0,1), (3,0,0)], 
                # J7
                [(0,0,1), (1,0,0), (1,0,1), (1,1,1), (2,0,0), (2,0,1), (2,1,0)], 
                # L7 
                [(0,0,0), (0,0,1), (1,0,0), (1,0,1), (2,0,0), (2,0,1), (2,1,0)],
                # M7 
                [(0,1,1), (1,0,1), (1,1,0), (1,1,1), (2,0,1), (2,1,0), (2,1,1)], 
                # N7
                [(0,1,1), (0,2,0), (1,1,0), (1,1,1), (2,0,1), (2,1,0), (3,0,0)],
                # P7
                [(0,0,0), (0,0,1), (0,1,0), (0,1,1), (0,2,0), (0,2,1), (1,0,0)], 
                # Q7
                [(0,1,1), (1,0,1), (1,1,0), (1,1,1), (1,2,0), (1,2,1), (2,2,0)], 
                # R7
                [(0,1,1), (1,0,1), (1,1,0), (2,0,0), (2,0,1), (3,0,0), (3,0,1)], 
                # S7
                [(0,2,0), (0,2,1), (1,0,1), (1,1,0), (1,1,1), (1,2,0), (2,0,0)],
                # T7
                [(0,1,1), (1,0,1), (1,1,0), (1,1,1), (1,2,0), (1,2,1), (2,0,0)], 
                # U7 
                [(0,1,1), (1,0,1), (1,1,0), (1,1,1), (2,0,1), (2,1,0), (3,0,0)],
                # V7 
                [(0,0,0), (0,0,1), (0,1,0), (0,1,1), (1,0,1), (1,1,0), (2,0,0)], 
                # W7 
                [(0,1,1), (1,1,0), (1,1,1), (1,2,0), (1,2,1), (2,0,1), (2,1,0)],
                # X7 
                [(0,2,1), (1,1,0), (1,1,1), (1,2,0), (1,2,1), (2,0,1), (2,1,0)], 
                # Y7 
                [(0,2,1), (1,0,1), (1,1,0), (1,1,1), (1,2,0), (2,0,1), (2,1,0)], 
                # Z7
                [(0,0,1), (0,1,1), (1,0,0), (1,0,1), (1,1,0), (1,1,1), (2,1,0)]
            ]
            shapes_name += ["A7", "B7", "C7", "D7", "E7", "F7", "G7", "H7", "I7", "J7", "L7", "M7", "N7", "P7", "Q7", "R7", "X7", "Y7", "Z7"]
        if size == 6: 
            shapes += [
            # rhomboid: I6
            [
                (0, 0, 0), (0, 0, 1), (1, 0, 0), (1, 0, 1), (2, 0, 0), (2, 0, 1)
            ], 
            # club: J6
            [
                (0, 0, 1), (1, 0, 0), (1, 0, 1), (2, 0, 0), (2, 0, 1), (0, 1, 0)
            ],
            # Crown: E6
            [   
                (0, 0, 1), (1, 0, 0), (1, 0, 1), (2, 0, 0), (2, 0, 1), (1, 1, 0)
            ],
            # Sphinx: P6
            [
                (0, 0, 0), (0, 0, 1), (1, 0, 0), (1, 0, 1), (2, 0, 0), (0, 1, 0)
            ],
            # Snake: S6
            [   
                (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 1, 0), (1, 1, 1), (1, 2, 0)
            ],
            # Yacht: F6
            [   
                (0, 0, 0), (0, 0, 1), (1, 0, 0), (1, 0, 1), (0, 1, 0), (1, 1, 0)
            ],
            # Bat: C6
            [
                (0, 0, 0), (0, 0, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)
            ],
            # Pistol: H6
            [
                (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 1, 0), (1, 1, 1), (0, 2, 0)
            ],
            # Lobster: V6
            [
                (0, 0, 1), (0, 1, 0), (0, 1, 1), (0, 2, 0), (1, 0, 1), (1, 1, 0)
            ],
            # Hook: G6
            [
                (0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 1), (1, 1, 0)
            ],
            # Hexagon: O6
            [
                (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0)
            ],
            # Butterfly: X6
            [
                (0, 1, 1), (1, 0, 1), (1, 0, 0), (2, 0, 0), (1, 1, 0), (1, 1, 1)
            ]
        ]
            shapes_name += ["I6", "J6", "E6", "P6", "S6", "F6", "C6", "H6", "V6", "G6", "O6", "X6"]
        if size == 5:
            shapes += [
                [
                    (0, 0, 0), (0, 0, 1), (1, 0, 0), (1, 0, 1), (2, 0, 0)
                ],
                [
                    (0, 0, 0), (0, 0, 1), (0, 1, 0), (1, 0, 0), (1, 0, 1)
                ],
                [
                    (0, 0, 0), (0, 0, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0)
                ],
                [
                    (0, 0, 1), (0, 1, 0), (1, 0, 0), (1, 0, 1), (1, 1, 0)
                ],
            ]
        if size == 4:
            shapes += [
                [
                    (0, 0, 0), (0, 0, 1), (1, 0, 0), (1, 0, 1)
                ],
                [
                    (0, 0, 0), (0, 0, 1), (0, 1, 0), (1, 0, 0)
                ],
                [
                    (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 1, 0)
                ],
            ]
        if size == 3:
            shapes += [
                [
                    (0, 0, 0), (0, 0, 1), (1, 0, 0)
                ]
            ]
        if size == 2:
            shapes += [
            [
                (0, 0, 0), (0, 0, 1)
            ]
        ]
        if size == 1:
            shapes += [
                [
                    (0, 0, 0)
                ]
            ]
    return shapes


if __name__ == "__main__":
    temp_cands = get_polyiamonds_by_size([7])
    print(len(temp_cands))
    

24


In [22]:
def generate_grid(name = "default", raw_grid = None):
    if name == "4x9":
        return {
            "x": 9, 
            "y": 4,
            "grid": ",".join([f"{a}_{b}_{c}" for a in range(9) for b in range(4) for c in range(2)])
        }
    if name == "6x6":
        return {
            "x": 6,
            "y": 6,
            "grid": ",".join([f"{a}_{b}_{c}" for a in range(6) for b in range(6) for c in range(2)])
        }
    if name == "7x12": 
        return {
            "x": 12,
            "y": 7, 
            "grid": ",".join([f"{a}_{b}_{c}" for a in range(12) for b in range(7) for c in range(2)])
        }
    if raw_grid: 
        # parse grid to standard format!
        raw_grid = clear_padding(raw_grid) 
        max_x = 0
        max_y = 0
        for (x, y, z) in raw_grid:
            max_x = max(max_x, x)
            max_y = max(max_y, y)
        return {
            "x": max_x + 1, 
            "y": max_y + 1,
            "grid": ",".join([f"{x}_{y}_{z}" for (x, y, z) in raw_grid])
        }
    else:
        return {}
        
        
if __name__ == "__main__":
    test_grid = generate_grid(name = "7x12")
    

In [24]:

class Grid:
    def __init__(self, grid_dict):
        self.width = grid_dict['x']
        self.height = grid_dict['y']
        self.grid_str = grid_dict['grid'] 
        self.positions = set(tuple(int(part) for part in coord.split('_')) for coord in self.grid_str.split(','))
        self.cand_shapes = grid_dict['cand_shapes']
        self.model = cp.CpModel() 
        self.solver = cp.CpSolver()
        self.x = dict()  # variables
        self.avail_variables = defaultdict(list) # record available variables of each cell
        self.shape_type = defaultdict(list)
        
    def get_all_feasible_pos(self, shape): 
        candidates = set()
        for x_1 in range(self.width):
            for y_1 in range(self.height):
                if self.check_fit(shape, x_1, y_1):
                    candidates.add((x_1, y_1))
        return candidates

    def check_fit(self, shape, x_, y_):
        for (x, y, z) in shape: 
            if (x + x_, y + y_, z) not in self.positions:
                return False 
        return True

    def add_all_vars(self): 
        for shape_name, shape in self.cand_shapes.items(): 
            temp_type, temp_index = shape_name[0], shape_name[1]
            
            cur_cand_pos = self.get_all_feasible_pos(shape) 
            
            for (x_1, y_1) in cur_cand_pos:
                self.x[f"{temp_type}_{temp_index}_{x_1}_{y_1}"] = self.model.NewBoolVar(f"{temp_type}_{temp_index}_{x_1}_{y_1}")
                for (x_2, y_2, z_2) in shape: 
                    # Exact Cover must start from the grid!
                    self.avail_variables[x_1 + x_2, y_1 + y_2, z_2].append(self.x[f"{temp_type}_{temp_index}_{x_1}_{y_1}"])
                self.shape_type[temp_type].append(self.x[f"{temp_type}_{temp_index}_{x_1}_{y_1}"])
        
        for temp_type, temp_vars in self.shape_type.items():
            self.model.Add(sum(temp_vars) == 1 )
            pass


        for k, avail_vars in self.avail_variables.items(): 
            # Adjust here!!!!
            if (int(k[0]), int(k[1]), int(k[2])) in self.positions: 
                self.model.Add(sum(avail_vars) == 1) 
            # each cell must be filled exactly once.


    def solve(self): 
        self.add_all_vars() 
        status = self.solver.Solve(self.model)
        result = []
        if status == cp.OPTIMAL:
            print("Found!")
            for k_ in self.x.keys(): 
                if self.solver.Value(self.x[k_]) > 1e-5: 
                    shape_info = list(map(int, k_.split("_")))
                    cur_shape = self.cand_shapes[shape_info[0], shape_info[1]]
                    cur_res = []
                    for (x_2, y_2, z_2) in cur_shape: 
                        cur_res.append([x_2 + shape_info[2], y_2 + shape_info[3], z_2])
                    result.append(cur_res)
            print(result)
        
        else:
            print("Can't find Optimal.")
        return result


    
if __name__ == "__main__":

    import json
    classic_shapes = None
    with open("../assets/data/Polyiamond/default_shapes.json", 'r') as f:
        classic_shapes = json.load(f)

    raw_grid = classic_shapes['snowflake-2']
    test_grid = generate_grid(name = "diamond", raw_grid=raw_grid)

    cand_shape = get_polyiamonds_by_size([7])

    
    all_cand_shapes = dict()
    for idx, shape in enumerate(cand_shape):  

        new_shapes = all_possible_rotations(shape)

        new_shapes = remove_dup(new_shapes)

        for idx_2, new_shape in enumerate(new_shapes): 
            all_cand_shapes[idx, idx_2] = new_shape
            # with rotation and flip ... 
    test_grid['cand_shapes'] = all_cand_shapes
    dummy_grid = Grid(test_grid)
    result = dummy_grid.solve()

    # test_candidates = dummy_grid.get_all_feasible_pos(triangles)

Found!
[[[5, 7, 1], [5, 8, 1], [5, 9, 0], [6, 7, 0], [6, 7, 1], [6, 8, 0], [6, 8, 1]], [[8, 7, 0], [9, 7, 0], [8, 6, 1], [9, 6, 0], [9, 6, 1], [10, 6, 0], [9, 5, 1]], [[10, 7, 1], [11, 7, 0], [11, 6, 1], [11, 6, 0], [11, 5, 1], [10, 5, 1], [11, 5, 0]], [[11, 11, 1], [11, 11, 0], [10, 11, 1], [10, 11, 0], [11, 10, 1], [11, 10, 0], [10, 10, 1]], [[15, 4, 0], [14, 4, 1], [14, 5, 0], [13, 5, 1], [13, 6, 0], [12, 6, 1], [13, 5, 0]], [[6, 12, 1], [6, 12, 0], [5, 12, 1], [5, 12, 0], [4, 12, 1], [6, 13, 0], [5, 13, 0]], [[4, 15, 0], [4, 14, 1], [5, 14, 0], [5, 13, 1], [4, 14, 0], [4, 13, 1], [4, 13, 0]], [[2, 9, 1], [3, 9, 0], [3, 9, 1], [4, 9, 0], [4, 8, 1], [5, 8, 0], [4, 9, 1]], [[2, 10, 0], [2, 10, 1], [3, 10, 0], [3, 10, 1], [4, 10, 0], [4, 10, 1], [5, 10, 0]], [[10, 6, 1], [10, 7, 0], [9, 7, 1], [8, 7, 1], [9, 8, 0], [8, 8, 1], [8, 8, 0]], [[9, 9, 0], [8, 9, 1], [8, 10, 0], [7, 10, 1], [7, 11, 0], [6, 11, 1], [6, 11, 0]], [[11, 2, 1], [10, 2, 1], [11, 2, 0], [11, 1, 1], [10, 1, 1], [11, 